## 2.3. Setup Function Tools for ReAct Agent

In [47]:
from langchain_core.tools import tool

#Tool annotation identifies a function as a tool automatically
@tool
def find_sum(x:int, y:int) -> int :
    #The docstring comment describes the capabilities of the function
    #It is used by the agent to discover the function's inputs, outputs and capabilities
    """
    This function is used to add two numbers and return their sum.
    It takes two integers as inputs and returns an integer as output.
    """
    return x + y

@tool
def find_product(x:int, y:int) -> int :
    """
    This function is used to multiply two numbers and return their product.
    It takes two integers as inputs and returns an integer as ouput.
    """
    return x * y


## 2.4. Create a basic ReAct Agent

In [48]:
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
# Setup the LLM for the agent

# Setup the LLM
# model = ChatOllama(model="llama3.2")
# model = ChatOllama(model="phi4-mini")
model = ChatOpenAI(model="gpt-5")

# Setup the GPT model

# Test the model
# response = model.invoke("Hello, how are you?")
# print(response.content)

In [54]:
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage,HumanMessage,SystemMessage

#Create list of tools available to the agent
agent_tools=[find_sum, find_product]

#System prompt
system_prompt = SystemMessage(
    """You are a Math genius who can solve math problems. Solve the
    problems provided by the user, by using only tools available.
    Do not solve the problem yourself"""
)

agent_graph=create_react_agent(
    model=model,
    prompt=system_prompt,
    tools=agent_tools,
    debug=True)


## 2.5. Execute the ReAct Agent

In [56]:
#Example 1
inputs = {"messages":[("user","what is the sum of 2 and 3? Do it twice and verify if the answer is the same. Explain step by step to primary school student.")]}

result = agent_graph.invoke(inputs)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

[values] {'messages': [HumanMessage(content='what is the sum of 2 and 3? Do it twice and verify if the answer is the same. Explain step by step to primary school student.', additional_kwargs={}, response_metadata={}, id='795fea7b-d160-4ed2-8387-4dae9d3f3abb')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1XSYXzqN2FNIRCdKqUFAPCQv', 'function': {'arguments': '{"x": 2, "y": 3}', 'name': 'find_sum'}, 'type': 'function'}, {'id': 'call_B4kqhi9Rb70UcNqD5ZINFi1j', 'function': {'arguments': '{"x": 2, "y": 3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 253, 'prompt_tokens': 263, 'total_tokens': 516, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 192, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None,

In [51]:
from pprint import pprint

#Example 2
inputs = {"messages":[("user","What is 3 multiply by 2, then add 10, then multiply by 11? Do it step by step, using the tools available.")]}

result = agent_graph.invoke(inputs)
pprint(result)

#Get the final answer
print(f"Agent returned : {result['messages'][-1].content} \n")

print("Step by Step execution : ")
for message in result['messages']:
    print(message.pretty_repr())

{'messages': [HumanMessage(content='What is 3 multiply by 2, then add 10, then multiply by 11? Do it step by step, using the tools available.', additional_kwargs={}, response_metadata={}, id='c3a25416-71bc-48e4-9c26-87d7f84b3350'),
              AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mqvZGJ2kRLMt1IvKvALBxvVW', 'function': {'arguments': '{"x": 3, "y": 2}', 'name': 'find_product'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 478, 'prompt_tokens': 262, 'total_tokens': 740, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CHLpy1wNSavr6GTtFujvkLtopT6Of', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--d0748d85-e480-4a8b-b3e2-3aa2bffb3849-0'

## 2.6. Debugging the Agent

In [52]:
agent_graph=create_react_agent(
    model=model,
    prompt=system_prompt,
    tools=agent_tools,
    debug=True)

inputs = {"messages":[("user","what is the sum of 2 and 3 ?")]}

result = agent_graph.invoke(inputs)

[values] {'messages': [HumanMessage(content='what is the sum of 2 and 3 ?', additional_kwargs={}, response_metadata={}, id='b2d27c1a-983d-496e-b0f3-5bc5a1ced67e')]}
[updates] {'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_9fyGhvLzs0iuJzuLrLsB7tws', 'function': {'arguments': '{"x":2,"y":3}', 'name': 'find_sum'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 155, 'prompt_tokens': 242, 'total_tokens': 397, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CHLq8jKTEm9z8bQBb8cSev1yqYfVY', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--fa4bf8f3-e3c9-4ed2-9951-addf5aa49257-0', tool_calls=[{'name': 'find_sum', 'args': {'x': 2, 'y